# LSTM

In [31]:
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd

import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 20
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['Adiac']
each = 'Adiac'
#for each in flist:
fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)


Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

In [32]:
# 先整理一下数据格式
#timesteps = 1
#x_train = x_train.reshape(x_train.shape[0], timesteps, x_train.shape[1])
#x_test = x_test.reshape(x_test.shape[0], timesteps, x_test.shape[1])

print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

x_train[0]

# x_train.shape:  (390, 176)
# x_test.shape:  (391, 176)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


array([ 1.60255907,  1.60396307,  1.57498062,  1.55492356,  1.5117006 ,
        1.43839205,  1.37290575,  1.30902402,  1.21375299,  1.11988595,
        1.02692149,  0.92862184,  0.83047262,  0.74132902,  0.64488466,
        0.55801754,  0.46427084,  0.37063446,  0.27922442,  0.18636023,
        0.09580462,  0.01067644, -0.0810866 , -0.16602223, -0.24418459,
       -0.33638689, -0.42732559, -0.49890924, -0.56846712, -0.65171394,
       -0.73433899, -0.78001895, -0.8453849 , -0.91991693, -0.99078855,
       -1.05800978, -1.09992903, -1.15037254, -1.20181889, -1.24674671,
       -1.28525626, -1.3163447 , -1.3395106 , -1.35495454, -1.38082814,
       -1.39185953, -1.38082814, -1.36097166, -1.3333932 , -1.29739078,
       -1.26128807, -1.23391019, -1.17754985, -1.11737868, -1.07505828,
       -1.00967227, -0.95163717, -0.87265247, -0.79242424, -0.72657691,
       -0.64467391, -0.59331781, -0.52287742, -0.43185849, -0.35185088,
       -0.27062982, -0.17413531, -0.08936716, -0.00393141,  0.08

In [4]:
# 使用LSTM的序列分类
   
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(input_dim=x_train.shape[0], output_dim=256, input_length=x_train.shape[1]))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
    
#    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
#    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5, patience=50, min_lr=0.0001) 
#    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
#              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
model.fit(x_train, Y_train, batch_size = batch_size, epochs = nb_epochs)
score = model.evaluate(x_test, Y_test, batch_size = batch_size)
print(score)

#     hist = model.fit(x_train,Y_train,batch_size=batch_size,nb_epoch=nb_epochs,verbose=1,validation_data=(x_test, Y_test),callbacks = [reduce_lr])

#     Print the testing results which has the lowest training loss.
#    log = pd.DataFrame(hist.history)
#    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

NameError: name 'x_train' is not defined

# 栈式LSTM

In [6]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 30 20:11:19 2016

@author: stephen
"""
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd

import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 1000
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

#flist  = ['Phoneme']
#for each in flist:
each = 'Adiac'
fname = each

x_train, y_train = readucr(foder + fname + '/' + fname + '_TRAIN')
x_test, y_test = readucr(foder + fname + '/' + fname + '_TEST')
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)


Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)


# 先整理一下数据格式
timesteps = 1
x_train = x_train.reshape(x_train.shape[0], timesteps, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], timesteps, x_test.shape[1])

print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 1, 176)
# x_test.shape:  (391, 1, 176)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


In [5]:
# 用于序列分类的栈式LSTM

from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = nb_feature
timesteps = 1
num_classes = nb_classes

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(128, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(64, return_sequences=True))  # returns a sequence of vectors of dimension 32

model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,validation_data=(x_test, Y_test))

#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print('\n','min_loss: ',log.loc[log['loss'].idxmin]['loss'], 'val_acc: ',log.loc[log['loss'].idxmin]['val_acc'])

Train on 390 samples, validate on 391 samples
Epoch 1/1000
390/390 [==============================] - 2s - loss: 3.6105 - acc: 0.0179 - val_loss: 3.6071 - val_acc: 0.0460
Epoch 2/1000
390/390 [==============================] - 0s - loss: 3.5860 - acc: 0.0667 - val_loss: 3.5746 - val_acc: 0.0767
Epoch 3/1000
390/390 [==============================] - 0s - loss: 3.5250 - acc: 0.0949 - val_loss: 3.5121 - val_acc: 0.0870
Epoch 4/1000
390/390 [==============================] - 0s - loss: 3.4456 - acc: 0.1000 - val_loss: 3.4644 - val_acc: 0.1074
Epoch 5/1000
390/390 [==============================] - 0s - loss: 3.3902 - acc: 0.0949 - val_loss: 3.4837 - val_acc: 0.0716
Epoch 6/1000
390/390 [==============================] - 0s - loss: 3.3462 - acc: 0.1077 - val_loss: 3.3863 - val_acc: 0.0972
Epoch 7/1000
390/390 [==============================] - 0s - loss: 3.3088 - acc: 0.1026 - val_loss: 3.3582 - val_acc: 0.1176
Epoch 8/1000
390/390 [==============================] - 0s - loss: 3.2851 - acc

390/390 [==============================] - 0s - loss: 2.1082 - acc: 0.3436 - val_loss: 2.2876 - val_acc: 0.3146
Epoch 66/1000
390/390 [==============================] - 0s - loss: 2.0614 - acc: 0.3821 - val_loss: 2.2880 - val_acc: 0.2890
Epoch 67/1000
390/390 [==============================] - 0s - loss: 2.0419 - acc: 0.3821 - val_loss: 2.4084 - val_acc: 0.2737
Epoch 68/1000
390/390 [==============================] - 0s - loss: 2.0708 - acc: 0.3564 - val_loss: 2.2692 - val_acc: 0.3197
Epoch 69/1000
390/390 [==============================] - 0s - loss: 2.0474 - acc: 0.3769 - val_loss: 2.2732 - val_acc: 0.3043
Epoch 70/1000
390/390 [==============================] - 0s - loss: 2.0392 - acc: 0.3744 - val_loss: 2.4019 - val_acc: 0.2609
Epoch 71/1000
390/390 [==============================] - 0s - loss: 2.0033 - acc: 0.3974 - val_loss: 2.4034 - val_acc: 0.2609
Epoch 72/1000
390/390 [==============================] - 0s - loss: 1.9829 - acc: 0.4103 - val_loss: 2.2841 - val_acc: 0.2916
Epoch 

390/390 [==============================] - 0s - loss: 1.4906 - acc: 0.5333 - val_loss: 1.9682 - val_acc: 0.4015
Epoch 130/1000
390/390 [==============================] - 0s - loss: 1.4833 - acc: 0.5282 - val_loss: 1.9590 - val_acc: 0.3862
Epoch 131/1000
390/390 [==============================] - 0s - loss: 1.4768 - acc: 0.5205 - val_loss: 1.7647 - val_acc: 0.4655
Epoch 132/1000
390/390 [==============================] - 0s - loss: 1.4795 - acc: 0.5564 - val_loss: 2.2627 - val_acc: 0.3504
Epoch 133/1000
390/390 [==============================] - 0s - loss: 1.4849 - acc: 0.5359 - val_loss: 1.9854 - val_acc: 0.3887
Epoch 134/1000
390/390 [==============================] - 0s - loss: 1.4891 - acc: 0.5564 - val_loss: 1.6769 - val_acc: 0.4655
Epoch 135/1000
390/390 [==============================] - 0s - loss: 1.4290 - acc: 0.5462 - val_loss: 1.9260 - val_acc: 0.3964
Epoch 136/1000
390/390 [==============================] - 0s - loss: 1.4340 - acc: 0.5692 - val_loss: 2.2561 - val_acc: 0.3043

390/390 [==============================] - 0s - loss: 1.1129 - acc: 0.6385 - val_loss: 1.6539 - val_acc: 0.4885
Epoch 194/1000
390/390 [==============================] - 0s - loss: 1.0841 - acc: 0.6410 - val_loss: 2.3745 - val_acc: 0.3811
Epoch 195/1000
390/390 [==============================] - 0s - loss: 1.1173 - acc: 0.6410 - val_loss: 1.6474 - val_acc: 0.4527
Epoch 196/1000
390/390 [==============================] - 0s - loss: 1.1265 - acc: 0.6103 - val_loss: 1.5741 - val_acc: 0.5090
Epoch 197/1000
390/390 [==============================] - 0s - loss: 1.1080 - acc: 0.6385 - val_loss: 1.7493 - val_acc: 0.4425
Epoch 198/1000
390/390 [==============================] - 0s - loss: 1.0477 - acc: 0.6641 - val_loss: 1.7827 - val_acc: 0.4118
Epoch 199/1000
390/390 [==============================] - 0s - loss: 1.0525 - acc: 0.6744 - val_loss: 1.6294 - val_acc: 0.5090
Epoch 200/1000
390/390 [==============================] - 0s - loss: 1.1156 - acc: 0.6410 - val_loss: 1.6582 - val_acc: 0.4859

390/390 [==============================] - 0s - loss: 0.9239 - acc: 0.6949 - val_loss: 1.4961 - val_acc: 0.5448
Epoch 258/1000
390/390 [==============================] - 0s - loss: 0.8627 - acc: 0.7000 - val_loss: 1.7782 - val_acc: 0.4604
Epoch 259/1000
390/390 [==============================] - 0s - loss: 0.9801 - acc: 0.6718 - val_loss: 1.6910 - val_acc: 0.5090
Epoch 260/1000
390/390 [==============================] - 0s - loss: 0.8644 - acc: 0.7256 - val_loss: 1.9131 - val_acc: 0.4118
Epoch 261/1000
390/390 [==============================] - 0s - loss: 0.8879 - acc: 0.7179 - val_loss: 1.5104 - val_acc: 0.5269
Epoch 262/1000
390/390 [==============================] - 0s - loss: 0.9442 - acc: 0.6897 - val_loss: 1.4951 - val_acc: 0.5729
Epoch 263/1000
390/390 [==============================] - 0s - loss: 0.8500 - acc: 0.7205 - val_loss: 1.9131 - val_acc: 0.4066
Epoch 264/1000
390/390 [==============================] - 0s - loss: 0.8898 - acc: 0.6923 - val_loss: 1.4652 - val_acc: 0.5831

...

390/390 [==============================] - 0s - loss: 0.2758 - acc: 0.9154 - val_loss: 1.8450 - val_acc: 0.6036
Epoch 706/1000
390/390 [==============================] - 0s - loss: 0.2988 - acc: 0.9051 - val_loss: 1.9343 - val_acc: 0.5601
Epoch 707/1000
390/390 [==============================] - 0s - loss: 0.2653 - acc: 0.9077 - val_loss: 1.8811 - val_acc: 0.5754
Epoch 708/1000
390/390 [==============================] - 0s - loss: 0.2841 - acc: 0.8974 - val_loss: 2.1079 - val_acc: 0.5269
Epoch 709/1000
390/390 [==============================] - 0s - loss: 0.2804 - acc: 0.9000 - val_loss: 1.8726 - val_acc: 0.5831
Epoch 710/1000
390/390 [==============================] - 0s - loss: 0.2898 - acc: 0.9026 - val_loss: 1.9766 - val_acc: 0.5345
Epoch 711/1000
390/390 [==============================] - 0s - loss: 0.2684 - acc: 0.8974 - val_loss: 2.3939 - val_acc: 0.5141
Epoch 712/1000
390/390 [==============================] - 0s - loss: 0.2063 - acc: 0.9487 - val_loss: 1.9062 - val_acc: 0.5703

390/390 [==============================] - 0s - loss: 0.2462 - acc: 0.9179 - val_loss: 2.0873 - val_acc: 0.5166
Epoch 770/1000
390/390 [==============================] - 0s - loss: 0.2590 - acc: 0.9359 - val_loss: 1.6905 - val_acc: 0.6445
Epoch 771/1000
390/390 [==============================] - 0s - loss: 0.2346 - acc: 0.9359 - val_loss: 1.7940 - val_acc: 0.6087
Epoch 772/1000
390/390 [==============================] - 0s - loss: 0.2491 - acc: 0.9128 - val_loss: 1.8154 - val_acc: 0.6010
Epoch 773/1000
390/390 [==============================] - 0s - loss: 0.3593 - acc: 0.8846 - val_loss: 1.7231 - val_acc: 0.6113
Epoch 774/1000
390/390 [==============================] - 0s - loss: 0.2139 - acc: 0.9385 - val_loss: 2.4367 - val_acc: 0.4578
Epoch 775/1000
390/390 [==============================] - 0s - loss: 0.2813 - acc: 0.9077 - val_loss: 1.9971 - val_acc: 0.5703
Epoch 776/1000
390/390 [==============================] - 0s - loss: 0.2812 - acc: 0.9103 - val_loss: 1.6693 - val_acc: 0.6164

390/390 [==============================] - 0s - loss: 0.2736 - acc: 0.9256 - val_loss: 1.8452 - val_acc: 0.6138
Epoch 834/1000
390/390 [==============================] - 0s - loss: 0.3006 - acc: 0.9103 - val_loss: 1.7768 - val_acc: 0.6240
Epoch 835/1000
390/390 [==============================] - 0s - loss: 0.1636 - acc: 0.9564 - val_loss: 2.3811 - val_acc: 0.4910
Epoch 836/1000
390/390 [==============================] - 0s - loss: 0.2701 - acc: 0.9077 - val_loss: 1.7980 - val_acc: 0.6215
Epoch 837/1000
390/390 [==============================] - 0s - loss: 0.2306 - acc: 0.9256 - val_loss: 1.8437 - val_acc: 0.6189
Epoch 838/1000
390/390 [==============================] - 0s - loss: 0.1789 - acc: 0.9487 - val_loss: 2.6582 - val_acc: 0.4757
Epoch 839/1000
390/390 [==============================] - 0s - loss: 0.3031 - acc: 0.8872 - val_loss: 2.0519 - val_acc: 0.5192
Epoch 840/1000
390/390 [==============================] - 0s - loss: 0.2628 - acc: 0.9051 - val_loss: 1.9406 - val_acc: 0.5754

390/390 [==============================] - 0s - loss: 0.2168 - acc: 0.9231 - val_loss: 1.8584 - val_acc: 0.6138
Epoch 898/1000
390/390 [==============================] - 0s - loss: 0.1653 - acc: 0.9590 - val_loss: 1.8890 - val_acc: 0.6061
Epoch 899/1000
390/390 [==============================] - 0s - loss: 0.2295 - acc: 0.9205 - val_loss: 2.3303 - val_acc: 0.5243
Epoch 900/1000
390/390 [==============================] - 0s - loss: 0.2049 - acc: 0.9385 - val_loss: 2.2321 - val_acc: 0.5627
Epoch 901/1000
390/390 [==============================] - 0s - loss: 0.1505 - acc: 0.9590 - val_loss: 2.1845 - val_acc: 0.50640.979
Epoch 902/1000
390/390 [==============================] - 0s - loss: 0.2519 - acc: 0.9282 - val_loss: 2.4520 - val_acc: 0.4962
Epoch 903/1000
390/390 [==============================] - 0s - loss: 0.2012 - acc: 0.9359 - val_loss: 2.0859 - val_acc: 0.5908
Epoch 904/1000
390/390 [==============================] - 0s - loss: 0.2217 - acc: 0.9256 - val_loss: 2.1917 - val_acc: 0

390/390 [==============================] - 0s - loss: 0.1852 - acc: 0.9436 - val_loss: 2.0267 - val_acc: 0.5831
Epoch 962/1000
390/390 [==============================] - 0s - loss: 0.1944 - acc: 0.9487 - val_loss: 2.1298 - val_acc: 0.5448
Epoch 963/1000
390/390 [==============================] - 0s - loss: 0.2308 - acc: 0.9359 - val_loss: 1.8737 - val_acc: 0.6138
Epoch 964/1000
390/390 [==============================] - ETA: 0s - loss: 0.2236 - acc: 0.930 - 0s - loss: 0.1822 - acc: 0.9436 - val_loss: 1.8562 - val_acc: 0.6215
Epoch 965/1000
390/390 [==============================] - 0s - loss: 0.2025 - acc: 0.9179 - val_loss: 1.8568 - val_acc: 0.6113
Epoch 966/1000
390/390 [==============================] - 0s - loss: 0.2485 - acc: 0.9128 - val_loss: 2.0172 - val_acc: 0.58310.927
Epoch 967/1000
390/390 [==============================] - 0s - loss: 0.1555 - acc: 0.9538 - val_loss: 2.3558 - val_acc: 0.5090
Epoch 968/1000
390/390 [==============================] - 0s - loss: 0.1495 - acc: 

# TensorFlow LSTM

In [ ]:
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd

import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 1000
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['50words']
for each in flist:
    fname = each
    x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    nb_feature = x_train.shape[1]
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)

    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std) 
    x_test = (x_test - x_train_mean)/(x_train_std)
    
    import tensorflow as tf
    import numpy as np
    from tensorflow.contrib import rnn
    
    '''1 首先设置好模型用到的各个超参数'''
    lr = 1e-3
    # 在训练和测试的时候，我们想用不同的 batch_size, 所以采用占位符的方式
    #batch_size = tf.placeholder(tf.int32)  # 注意类型必须为 tf.int32
    # 在 1.0 版本以后请使用：
    keep_prob = tf.placeholder(tf.float32,[])
    batch_size = tf.placeholder(tf.int32, [])
    
    # 每个时刻输入特征是 28 维的，就是每个时刻输入一行，一行有 28个像素
    input_size = 28
    # 时序持续长度为28，即每做一次预测，需要先输入 28 行
    timestep_size = 28
    # 每个隐含层的节点数
    hidden_size = 256
    # LSTM layer 的层数
    layer_num = 2
    # 最后输出分类类别数量，如果是回归预测的话应该是 1
    class_num = nb_classes
    
    _X = tf.placeholder(tf.float32, [None, nb_feature])
    y = tf.placeholder(tf.float32, [None, class_num])
    keep_prob = tf.placeholder(tf.float32)
    '''2 开始搭建LSTM模型，其实普通 RNNs 模型也一样'''
    # 我自己写的
    sess = tf.Session()
    #
    # 设置 GPU 按需增长
    #config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True
    #sess = tf.Session(config=config)
    
    # 把784个点的字符信息还原成 28*28 的图片
    # 下面几个步骤是实现 RNN / LSTM 的关键
    ######################################################################
    
    # **步骤1：RNN 的输入 shape = (batch_size, timestep_size, input_size)
    X = tf.reshape(_X, [-1, 28, 28])
    
    # 步骤2：定义一层 LSTM_cell, 只需要说明 hidden_size,它会自动匹配输入的 x 的维度
    lstm_cell = rnn.BasicLSTMCell(num_units=hidden_size, forget_bias=1.0, state_is_tuple=True, reuse=None)
    
    # 步骤3：添加 dropout layer，一般只设置 output_keep_prob
    lstm_cell = rnn.DropoutWrapper(cell=lstm_cell, input_keep_prob=1.0, output_keep_prob=keep_prob)
    
    # 步骤4：调用 MultiRNNCell 来实现多层 LSTM
    # mlstm_cell = rnn.MultiRNNCell([lstm_cell for _ in range(layer_num)], state_is_tuple=True)
    
    # 步骤5：用全零来初始化 state
    #init_state = mlstm_cell.zero_state(batch_size, dtype=tf.float32)
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
    
    # 步骤6：方法一，调用 dynamic_rnn() 来让我们构建好的网络运行起来
    # ** 当 time_major==False 时， outputs.shape = [batch_size, timestep_size, hidden_size] 
    # ** 所以，可以取 h_state = outputs[:, -1, :] 作为最后输出
    # ** state.shape = [layer_num, 2, batch_size, hidden_size], 
    # ** 或者，可以取 h_state = state[-1][1] 作为最后输出
    # ** 最后输出维度是 [batch_size, hidden_size]
    #outputs, state = tf.nn.dynamic_rnn(mlstm_cell, inputs=X, initial_state=init_state, time_major=False)
    outputs, state = tf.nn.dynamic_rnn(lstm_cell, inputs=X, initial_state=init_state, time_major=False)
    h_state = outputs[:, -1, :]  
    # 或者 
    #h_state = state[-1][1]
    
    # *************** 为了更好的理解 LSTM 工作原理，我们把上面 步骤6 中的函数自己来实现 ***************
    # 通过查看文档你会发现， RNNCell 都提供了一个 __call__()函数（见最后附），我们可以用它来展开实现LSTM按时间步迭代。
    # **步骤6：方法二，按时间步展开计算
    # outputs = list()
    # state = init_state
    # with tf.variable_scope('RNN'):
    #     for timestep in range(timestep_size):
    #         if timestep > 0:
    #             tf.get_variable_scope().reuse_variables()
    #         # 这里的state保存了每一层 LSTM 的状态
    #         (cell_output, state) = mlstm_cell(X[:, timestep, :], state)
    #         outputs.append(cell_output)
    # h_state = outputs[-1]
    # 
    
    '''3. 设置 loss function 和 优化器，展开训练并完成测试'''
    # 上面 LSTM 部分的输出会是一个 [hidden_size] 的tensor，我们要分类的话，还需要接一个 softmax 层
    # 首先定义 softmax 的连接权重矩阵和偏置
    # out_W = tf.placeholder(tf.float32, [hidden_size, class_num], name='out_Weights')
    # out_bias = tf.placeholder(tf.float32, [class_num], name='out_bias')
    # 开始训练和测试
    W = tf.Variable(tf.truncated_normal([hidden_size, class_num], stddev=0.1), dtype=tf.float32)
    bias = tf.Variable(tf.constant(0.1,shape=[class_num]), dtype=tf.float32)
    y_pre = tf.nn.softmax(tf.matmul(h_state, W) + bias)
    
    
    # 损失和评估函数
    cross_entropy = -tf.reduce_mean(y * tf.log(y_pre))
    train_op = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
    
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        _batch_size = 128
        batch = mnist.train.next_batch(_batch_size)
        if (i+1)%200 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={
                _X:batch[0], y: batch[1], keep_prob: 1.0, batch_size: _batch_size})
            # 已经迭代完成的 epoch 数: mnist.train.epochs_completed
            print("Iter%d, step %d, training accuracy %g" % ( mnist.train.epochs_completed, (i+1), train_accuracy))
        sess.run(train_op, feed_dict={_X: batch[0], y: batch[1], keep_prob: 0.5, batch_size: _batch_size})
    
    # 计算测试数据的准确率
    print("test accuracy %g"% sess.run(accuracy, feed_dict={
        _X: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0, batch_size:mnist.test.images.shape[0]}))

In [ ]:
print(x_train.shape)
print(Y_train.shape)

print(x_test.shape)
print(Y_test.shape)

In [48]:
x_train

array([[[-0.48184041, -0.43166584, -0.25955683, ..., -1.09214663,
         -1.09214663, -1.09214663]],

       [[-0.44651896, -0.42678525, -0.24963463, ..., -1.09093906,
         -1.09093906, -1.09093906]],

       [[-0.8509042 , -0.80036736, -0.64039444, ..., -0.84205874,
         -0.62311611, -0.42939159]],

       ..., 
       [[-0.71566634, -0.62715141, -0.47659754, ..., -0.90014289,
         -0.75640173, -0.6164141 ]],

       [[-0.53629178, -0.50434147, -0.43991758, ..., -1.01989365,
         -0.92272446, -0.74736507]],

       [[-0.56332124, -0.55906455, -0.38208501, ..., -0.83990524,
         -0.70222206, -0.7112889 ]]])

# ConvLSTM

In [35]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd

import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 100
nb_feature = 0
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
each = 'Adiac'
#for each in flist:
fname = each
x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
nb_classes = len(np.unique(y_test))
nb_feature = x_train.shape[1]
batch_size = min(x_train.shape[0]/10, 16)

y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)


Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean)/(x_train_std) 
x_test = (x_test - x_train_mean)/(x_train_std)

x_train = x_train.reshape( x_train.shape[0],1,1,1,x_train.shape[1])
x_test = x_test.reshape( x_test.shape[0],1,1,1,x_test.shape[1])

In [41]:
# ConvLSTM

from keras.models import Sequential
from keras.layers import LSTM, Dense, ConvLSTM2D
import numpy as np

data_dim = nb_feature
timesteps = 1
num_classes = nb_classes

#x = keras.layers.Input(x_train.shape[1:])

x = keras.layers.Input(x_train.shape[1:])

ConvLstm2D = keras.layers.ConvLSTM2D(16, (1,1), input_shape=(1,1,1,176), activation='relu', return_sequences=True, dropout=0.1)(x)

print('#########################################################')
print(ConvLstm2D)
print('#########################################################')

#full = keras.layers.pooling.GlobalAveragePooling2D()(ConvLstm2D)
    
#Lstm1 = LSTM(32, input_shape=(1,1,176,1))(x)  # returns a sequence of vectors of dimension 32

out = keras.layers.Dense(nb_classes, activation='softmax')(ConvLstm2D)

model = Model(input=x, output=out)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,validation_data=(x_test, Y_test))

#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print('\n','min_loss: ',log.loc[log['loss'].idxmin]['loss'], 'val_acc: ',log.loc[log['loss'].idxmin]['val_acc'])

#########################################################
Tensor("conv_lst_m2d_24/transpose_1:0", shape=(?, ?, 1, 1, 16), dtype=float32)
#########################################################


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


ValueError: Error when checking target: expected dense_7 to have 5 dimensions, but got array with shape (390, 37)

In [37]:
print("# x_train.shape: ",x_train.shape)
print("# x_test.shape: ",x_test.shape)

print("# Y_train.shape: ",Y_train.shape)
print("# Y_test.shape: ",Y_test.shape)

# x_train.shape:  (390, 1, 1, 1, 176)
# x_test.shape:  (391, 1, 1, 1, 176)
# Y_train.shape:  (390, 37)
# Y_test.shape:  (391, 37)


In [26]:
Y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

# 

In [5]:
# ConvLSTM

from keras.models import Sequential
from keras.layers import LSTM, Dense, ConvLSTM2D
import numpy as np

data_dim = nb_feature
timesteps = 1
num_classes = nb_classes


x = keras.layers.Input(x_train.shape[1:])

ConvLstm2D = keras.layers.ConvLSTM2D(16, (3,3), input_shape=(, x_train.shape[1:]),activation='relu', return_sequences=None, dropout=0.1)(x)
ConvLstm2D = keras.layers.Activation('relu')(ConvLstm2D)
out = keras.layers.Dense(nb_classes, activation='softmax')(ConvLstm2D)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs, validation_data=(x_test, Y_test))

#Print the testing results which has the lowest training loss.
log = pd.DataFrame(hist.history)
print('\n','min_loss: ',log.loc[log['loss'].idxmin]['loss'], 'val_acc: ',log.loc[log['loss'].idxmin]['val_acc'])

SyntaxError: invalid syntax (<ipython-input-5-d3841bfc8c32>, line 14)

In [85]:
x_train.shape

(390, 176, 1, 1)

In [101]:
x_train

array([[[ 1.60255907],
        [ 1.60396307],
        [ 1.57498062],
        ..., 
        [ 1.56866265],
        [ 1.57538176],
        [ 1.59744452]],

       [[ 1.70595321],
        [ 1.6753662 ],
        [ 1.6235187 ],
        ..., 
        [ 1.52403569],
        [ 1.60707191],
        [ 1.67496506]],

       [[ 1.7272137 ],
        [ 1.70013667],
        [ 1.66162711],
        ..., 
        [ 1.64648403],
        [ 1.69983581],
        [ 1.71337432]],

       ..., 
       [[ 2.60300517],
        [ 2.63258933],
        [ 2.43091561],
        ..., 
        [ 2.15011678],
        [ 2.1349737 ],
        [ 2.12364146]],

       [[ 1.66463567],
        [ 1.6487906 ],
        [ 1.59834709],
        ..., 
        [ 1.58440744],
        [ 1.64157005],
        [ 1.66493653]],

       [[ 1.73152596],
        [ 1.73383252],
        [ 1.71658345],
        ..., 
        [ 1.60095451],
        [ 1.64497975],
        [ 1.70364665]]])